In [ ]:
!pip install kagglehub tensorflow matplotlib


In [ ]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-dataset")

print("Path to dataset files:", path)


100%|██████████| 23.1G/23.1G [03:45<00:00, 110MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1


In [ ]:
!ls /root/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1


fashion-dataset


In [ ]:
!mkdir -p /content/fashion-dataset


In [ ]:
!mv /root/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-dataset/versions/1/* fashion-dataset/


In [ ]:
!ls /content/fashion-dataset


fashion-dataset


Data Loading

In [ ]:
import pandas as pd

# Load the styles.csv file
styles_csv_path = "/content/fashion-dataset/fashion-dataset/fashion-dataset/styles.csv"
styles_data = pd.read_csv(styles_csv_path, on_bad_lines='skip')

# Display the first few rows
print(styles_data.head())


      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName  
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  
1  2012.0  Casual             Peter England Men Party Blue Jeans  
2  2016.0  Casual                       Titan Women Silver Watch  
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  
4  2012.0  Casual                          Puma Men Grey T-shirt  


Preprocessing

In [ ]:
# Load and inspect the images.csv file
images_csv_path = "/content/fashion-dataset/fashion-dataset/fashion-dataset/images.csv"
images_data = pd.read_csv(images_csv_path)

# Display column names and preview the data
print("Columns in images.csv:", images_data.columns)
print(images_data.head())


Columns in images.csv: Index(['filename', 'link'], dtype='object')
    filename                                               link
0  15970.jpg  http://assets.myntassets.com/v1/images/style/p...
1  39386.jpg  http://assets.myntassets.com/v1/images/style/p...
2  59263.jpg  http://assets.myntassets.com/v1/images/style/p...
3  21379.jpg  http://assets.myntassets.com/v1/images/style/p...
4  53759.jpg  http://assets.myntassets.com/v1/images/style/p...


In [ ]:
# Extract 'id' from 'filename' in images.csv
images_data['id'] = images_data['filename'].str.extract(r'(\d+)', expand=False).astype(int)

# Display updated images_data
print(images_data.head())


    filename                                               link     id
0  15970.jpg  http://assets.myntassets.com/v1/images/style/p...  15970
1  39386.jpg  http://assets.myntassets.com/v1/images/style/p...  39386
2  59263.jpg  http://assets.myntassets.com/v1/images/style/p...  59263
3  21379.jpg  http://assets.myntassets.com/v1/images/style/p...  21379
4  53759.jpg  http://assets.myntassets.com/v1/images/style/p...  53759


In [ ]:
# Merge images.csv and styles.csv on 'id'
merged_data = pd.merge(styles_data, images_data, on='id', how='inner')

# Display the merged DataFrame
print(f"Merged data: {len(merged_data)} rows")
print(merged_data.head())


Merged data: 44424 rows
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName   filename  \
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  15970.jpg   
1  2012.0  Casual             Peter England Men Party Blue Jeans  39386.jpg   
2  2016.0  Casual                       Titan Women Silver Watch  59263.jpg   
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  21379.jpg   
4  2012.0  Casual                          Puma Men Grey T-shirt  53759.jpg   

                                

In [ ]:
import os

# Define the base directory for images
image_dir = "/content/fashion-dataset/fashion-dataset/fashion-dataset/images"

# Add full image paths
merged_data['full_path'] = merged_data['filename'].apply(lambda x: os.path.join(image_dir, x))

# Check if files exist
merged_data['file_exists'] = merged_data['full_path'].apply(os.path.exists)

# Filter valid rows with existing image files
valid_data = merged_data[merged_data['file_exists']]

print(f"Number of valid images: {len(valid_data)}")
print(valid_data.head())


Number of valid images: 44419
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName   filename  \
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  15970.jpg   
1  2012.0  Casual             Peter England Men Party Blue Jeans  39386.jpg   
2  2016.0  Casual                       Titan Women Silver Watch  59263.jpg   
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  21379.jpg   
4  2012.0  Casual                          Puma Men Grey T-shirt  53759.jpg   

                          

Train and Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Select relevant columns
data = valid_data[['full_path', 'subCategory']]

# Train-validation split
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")


Training samples: 35535, Validation samples: 8884


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode labels on the full dataset
all_labels = data['subCategory']
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Transform the labels for training and validation sets
train_data['label'] = label_encoder.transform(train_data['subCategory'])
val_data['label'] = label_encoder.transform(val_data['subCategory'])

# Save the mapping for reference
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"Label Mapping: {label_mapping}")



Label Mapping: {'Accessories': 0, 'Apparel Set': 1, 'Bags': 2, 'Bath and Body': 3, 'Beauty Accessories': 4, 'Belts': 5, 'Bottomwear': 6, 'Cufflinks': 7, 'Dress': 8, 'Eyes': 9, 'Eyewear': 10, 'Flip Flops': 11, 'Fragrance': 12, 'Free Gifts': 13, 'Gloves': 14, 'Hair': 15, 'Headwear': 16, 'Home Furnishing': 17, 'Innerwear': 18, 'Jewellery': 19, 'Lips': 20, 'Loungewear and Nightwear': 21, 'Makeup': 22, 'Mufflers': 23, 'Nails': 24, 'Perfumes': 25, 'Sandal': 26, 'Saree': 27, 'Scarves': 28, 'Shoe Accessories': 29, 'Shoes': 30, 'Skin': 31, 'Skin Care': 32, 'Socks': 33, 'Sports Accessories': 34, 'Sports Equipment': 35, 'Stoles': 36, 'Ties': 37, 'Topwear': 38, 'Umbrellas': 39, 'Vouchers': 40, 'Wallets': 41, 'Watches': 42, 'Water Bottle': 43, 'Wristbands': 44}


In [ ]:
print("Training data sample:")
print(train_data.head())

print("\nValidation data sample:")
print(val_data.head())


Training data sample:
                                               full_path subCategory  label
5041   /content/fashion-dataset/fashion-dataset/fashi...     Topwear     38
36282  /content/fashion-dataset/fashion-dataset/fashi...     Topwear     38
5882   /content/fashion-dataset/fashion-dataset/fashi...      Sandal     26
8736   /content/fashion-dataset/fashion-dataset/fashi...        Bags      2
5173   /content/fashion-dataset/fashion-dataset/fashi...  Bottomwear      6

Validation data sample:
                                               full_path subCategory  label
41944  /content/fashion-dataset/fashion-dataset/fashi...     Eyewear     10
24077  /content/fashion-dataset/fashion-dataset/fashi...     Watches     42
39613  /content/fashion-dataset/fashion-dataset/fashi...     Topwear     38
3150   /content/fashion-dataset/fashion-dataset/fashi...  Flip Flops     11
44249  /content/fashion-dataset/fashion-dataset/fashi...  Flip Flops     11


Creating Tensorflow

In [ ]:
import tensorflow as tf

# Define preprocessing function
def load_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Adjust for PNG if needed
    img = tf.image.resize(img, [128, 128])  # Resize to desired dimensions
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img, label

# Create TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_data['full_path'], train_data['label']))
train_ds = train_ds.map(load_image).batch(32).shuffle(buffer_size=1000)

val_ds = tf.data.Dataset.from_tensor_slices((val_data['full_path'], val_data['label']))
val_ds = val_ds.map(load_image).batch(32)

print("TensorFlow datasets created successfully!")


TensorFlow datasets created successfully!


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model with an Input layer
model = Sequential([
    Input(shape=(128, 128, 3)),  # Specify input shape here
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_mapping), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Save the training history for future analysis
print("Training complete!")


Epoch 1/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 712s 211ms/step - accuracy: 0.7989 - loss: 0.8148 - val_accuracy: 0.9068 - val_loss: 0.3510
Epoch 2/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 763s 197ms/step - accuracy: 0.9414 - loss: 0.2149 - val_accuracy: 0.9244 - val_loss: 0.2868
Epoch 3/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 754s 195ms/step - accuracy: 0.9658 - loss: 0.1104 - val_accuracy: 0.9346 - val_loss: 0.2981
Epoch 4/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 771s 189ms/step - accuracy: 0.9821 - loss: 0.0577 - val_accuracy: 0.9331 - val_loss: 0.3009
Epoch 5/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 709s 188ms/step - accuracy: 0.9896 - loss: 0.0358 - val_accuracy: 0.9287 - val_loss: 0.3758
Epoch 6/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 820s 251ms/step - accuracy: 0.9911 - loss: 0.0292 - val_accuracy: 0.9309 - val_loss: 0.4360
Epoch 7/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 779s 205ms/step - accuracy: 0.9924 - loss: 0.0256 - val_accuracy: 0.9347 - val_loss: 0.4025
Epoch 8/10
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 803s 249ms/step - ac

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(val_ds)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


278/278 ━━━━━━━━━━━━━━━━━━━━ 157s 564ms/step - accuracy: 0.9313 - loss: 0.5530
Validation Loss: 0.5463
Validation Accuracy: 0.9320


In [ ]:
# Save the model in Keras format
model.save('fashion_classifier.h5')
print("Model saved as fashion_classifier.h5")


Model saved as fashion_classifier.h5


In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('fashion_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved as fashion_classifier.tflite")


Saved artifact at '/tmp/tmphkhnw4__'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 45), dtype=tf.float32, name=None)
Captures:
  136568024110304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568023659120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568015831632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568015831280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568015846592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568015846944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568015832336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136568015405120: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model saved as fashion_classifier.tflite


Deployment of MOdel

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image


In [2]:
# Path to your .tflite model
model_path = "/content/fashion_classifier.tflite"

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Details:", input_details)
print("Output Details:", output_details)


Input Details: [{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([  1, 128, 128,   3], dtype=int32), 'shape_signature': array([ -1, 128, 128,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 17, 'shape': array([ 1, 45], dtype=int32), 'shape_signature': array([-1, 45], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [9]:
from google.colab import files
uploaded = files.upload()


Saving 10004.jpg to 10004.jpg


In [11]:
image_path = "/content/10004.jpg"  # Replace with your image name
image = Image.open(image_path).resize((128, 128))  # Resize to 128x128

# Convert to numpy array and normalize
input_image = np.array(image).astype(np.float32) / 255.0
input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension


In [12]:
# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_image)

# Run the model
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print output (confidence scores)
print("Model Output:", output_data)

# Find the predicted class
predicted_class = np.argmax(output_data)
print("Predicted Class:", predicted_class)


Model Output: [[1.07654665e-20 2.30723908e-23 3.36870992e-17 3.17822689e-28
  3.63732128e-24 1.84342822e-20 1.00000000e+00 4.21163371e-23
  7.81402628e-21 1.74176788e-16 1.37280509e-23 9.75474195e-27
  1.58863957e-17 3.11172277e-17 4.45801745e-17 8.72685886e-21
  8.45022564e-23 9.45690754e-31 1.81201507e-17 2.84445079e-19
  1.06444163e-15 1.44209926e-12 3.55715933e-16 6.62865010e-19
  2.47442947e-22 8.89223503e-27 7.33885110e-24 3.79228756e-23
  3.30145683e-20 6.24277893e-21 3.66615646e-17 2.39366349e-16
  5.32220511e-20 3.35247107e-15 7.67965962e-28 1.42499726e-26
  4.29837458e-22 1.94128445e-21 1.01030186e-17 2.18969019e-30
  5.39244956e-33 2.52480060e-16 1.45671371e-14 3.16463995e-21
  4.12067349e-28]]
Predicted Class: 6


In [13]:
print("Output Shape:", output_data.shape)
print("Output Data:", output_data)


Output Shape: (1, 45)
Output Data: [[1.07654665e-20 2.30723908e-23 3.36870992e-17 3.17822689e-28
  3.63732128e-24 1.84342822e-20 1.00000000e+00 4.21163371e-23
  7.81402628e-21 1.74176788e-16 1.37280509e-23 9.75474195e-27
  1.58863957e-17 3.11172277e-17 4.45801745e-17 8.72685886e-21
  8.45022564e-23 9.45690754e-31 1.81201507e-17 2.84445079e-19
  1.06444163e-15 1.44209926e-12 3.55715933e-16 6.62865010e-19
  2.47442947e-22 8.89223503e-27 7.33885110e-24 3.79228756e-23
  3.30145683e-20 6.24277893e-21 3.66615646e-17 2.39366349e-16
  5.32220511e-20 3.35247107e-15 7.67965962e-28 1.42499726e-26
  4.29837458e-22 1.94128445e-21 1.01030186e-17 2.18969019e-30
  5.39244956e-33 2.52480060e-16 1.45671371e-14 3.16463995e-21
  4.12067349e-28]]


In [14]:
predicted_class = np.argmax(output_data)
print("Predicted Class Index:", predicted_class)

if predicted_class < len(class_labels):
    print("Predicted Label:", class_labels[predicted_class])
else:
    print(f"Error: Predicted class index ({predicted_class}) is out of range.")


Predicted Class Index: 6
Error: Predicted class index (6) is out of range.


In [43]:
import tensorflow as tf
import numpy as np
from PIL import Image
from google.colab import files

# Path to your TensorFlow Lite model
model_path = "/content/fashion_classifier.tflite"

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Model Input Details:", input_details)
print("Model Output Details:", output_details)

# Correct class labels from your label mapping
class_labels = [
    'Accessories', 'Apparel Set', 'Bags', 'Bath and Body', 'Beauty Accessories',
    'Belts', 'Bottomwear', 'Cufflinks', 'Dress', 'Eyes', 'Eyewear', 'Flip Flops',
    'Fragrance', 'Free Gifts', 'Gloves', 'Hair', 'Headwear', 'Home Furnishing',
    'Innerwear', 'Jewellery', 'Lips', 'Loungewear and Nightwear', 'Makeup',
    'Mufflers', 'Nails', 'Perfumes', 'Sandal', 'Saree', 'Scarves',
    'Shoe Accessories', 'Shoes', 'Skin', 'Skin Care', 'Socks', 'Sports Accessories',
    'Sports Equipment', 'Stoles', 'Ties', 'Topwear', 'Umbrellas', 'Vouchers',
    'Wallets', 'Watches', 'Water Bottle', 'Wristbands'
]

# Single upload and classify
while True:
    print("Upload an image to classify (or type 'stop' to end):")
    uploaded = files.upload()  # Upload an image

    # Stop if no file is uploaded or the user types 'stop'
    if not uploaded:
        print("No file uploaded. Exiting...")
        break

    # Get the uploaded image file path
    image_path = list(uploaded.keys())[0]
    print(f"Classifying image: {image_path}")

    # Load and preprocess the image
    image = Image.open(image_path).resize((128, 128))  # Resize to match model input
    input_image = np.array(image).astype(np.float32) / 255.0  # Normalize pixel values
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_image)

    # Run inference
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Print confidence scores
    print("Confidence Scores:", output_data)

    # Find the predicted class
    predicted_class = np.argmax(output_data)
    print("Predicted Class Index:", predicted_class)

    # Ensure the predicted class index is valid
    if predicted_class < len(class_labels):
        predicted_label = class_labels[predicted_class]
        print(f"Predicted Label: {predicted_label}")
    else:
        print(f"Error: Predicted class index ({predicted_class}) is out of range.")

    print("-" * 50)  # Separator for next run
    break  # Stop after processing the first file


Model Input Details: [{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([  1, 128, 128,   3], dtype=int32), 'shape_signature': array([ -1, 128, 128,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Model Output Details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 17, 'shape': array([ 1, 45], dtype=int32), 'shape_signature': array([-1, 45], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Upload an image to classify (or type 'stop' to end):


Saving download (4).jpeg to download (4).jpeg
Classifying image: download (4).jpeg
Confidence Scores: [[2.1333010e-03 1.8490411e-01 2.2669262e-01 1.1797115e-05 4.3991677e-06
  3.5264794e-10 2.7757462e-02 3.6425900e-09 8.8166066e-02 9.7878173e-04
  9.5079253e-07 6.5646304e-08 2.2198159e-04 2.5281666e-02 1.8004463e-13
  2.9805386e-05 1.7879665e-06 5.2112193e-11 2.1672423e-07 1.9428082e-06
  2.2966717e-05 5.5759139e-03 1.0640382e-02 5.5643231e-05 1.7724724e-05
  9.9272626e-08 1.5795425e-05 9.2261347e-05 1.6655904e-04 1.0638660e-03
  7.5545181e-06 2.0887987e-03 3.7662074e-04 1.7704610e-03 2.3526833e-07
  6.2622767e-06 3.7039528e-04 5.2874483e-04 9.4222516e-02 3.9500745e-09
  9.9563930e-09 2.6195539e-07 5.7064623e-02 2.6972443e-01 8.9174171e-07]]
Predicted Class Index: 43
Predicted Label: Water Bottle
--------------------------------------------------
